<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [13]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [14]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login(api_key_value="wOWQmzzHeITT5wlJ.FkOButuQ3XpDXSUO1LnTuaNfD7SPWolfF1hateoistsLFFsBF7upULn5z6qKPOIB")
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
print(today)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163414
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
2024-11-21


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [15]:
# Retrieve feature groups

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [16]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,4.0,sweden,stockholm,Radarvagen,2024-11-21,https://api.waqi.info/feed/A58912


In [17]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [18]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-21,-3.05,0.0,21.995380,291.104919,stockholm
1,2024-11-22,-5.15,0.0,15.844090,268.698090,stockholm
2,2024-11-23,-2.90,0.0,8.640000,270.000000,stockholm
3,2024-11-24,3.90,0.0,15.463244,167.905258,stockholm
4,2024-11-25,7.25,0.3,18.775301,184.398621,stockholm
5,2024-11-26,5.70,0.0,12.434340,202.109497,stockholm
6,2024-11-27,5.60,0.0,3.671294,281.309906,stockholm
7,2024-11-28,1.40,0.0,12.641076,340.016815,stockholm
8,2024-11-29,-0.15,0.0,7.289445,327.094757,stockholm
9,2024-11-30,-0.20,0.0,5.091168,224.999893,stockholm


In [19]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [20]:
air_quality_df = air_quality_fg.read()
air_quality_df = air_quality_df.sort_values(by='date')
air_quality_df.tail(30)




Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


,date,pm25,pm25_3day_mean,country,city,street,url
855,2024-10-16 00:00:00+00:00,0.95,1.200000,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
333,2024-10-17 00:00:00+00:00,1.65,1.083333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1281,2024-10-18 00:00:00+00:00,4.78,1.133333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
816,2024-10-19 00:00:00+00:00,3.75,2.460000,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1043,2024-10-20 00:00:00+00:00,9.90,3.393333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
302,2024-10-21 00:00:00+00:00,3.33,6.143333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1257,2024-10-22 00:00:00+00:00,3.05,5.660000,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
772,2024-10-23 00:00:00+00:00,1.58,5.426666,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1476,2024-10-24 00:00:00+00:00,2.10,2.653333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1416,2024-10-25 00:00:00+00:00,2.67,2.243333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912


In [21]:
#Comment everything if not new feature
air_quality_df_copy1 = air_quality_df.copy()
air_quality_df_copy['date'] = pd.to_datetime(air_quality_df_copy1['date']).dt.date


air_quality_df = pd.concat([air_quality_df, aq_today_df], ignore_index=True)

air_quality_df['pm25_3day_mean'] = air_quality_df['pm25'].shift(1).rolling(window=3).mean()

air_quality_df.dropna(subset=['pm25_3day_mean'], inplace=True)
air_quality_df.at[air_quality_df.index[-1], 'date'] = air_quality_df['date'].iloc[-1].tz_localize(air_quality_df['date'].iloc[-2].tzinfo)

#air_quality_df.drop(air_quality_df.index[-1], inplace=True)


air_quality_df





,date,pm25,pm25_3day_mean,country,city,street,url
3,2019-12-18 00:00:00+00:00,0.750000,1.866667,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
4,2019-12-19 00:00:00+00:00,2.100000,1.683333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
5,2019-12-20 00:00:00+00:00,8.440000,1.953333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
6,2019-12-21 00:00:00+00:00,17.549999,3.763333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
7,2019-12-22 00:00:00+00:00,2.840000,9.363333,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
...,...,...,...,...,...,...,...
1791,2024-11-16 00:00:00+00:00,1.000000,5.666667,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1792,2024-11-17 00:00:00+00:00,3.000000,4.000000,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1793,2024-11-18 00:00:00+00:00,4.000000,3.666667,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912
1794,2024-11-20 00:00:00+00:00,4.000000,2.666667,sweden,stockholm,Radarvagen,https://api.waqi.info/feed/A58912


In [22]:

air_quality_df_copy = air_quality_df.copy()
air_quality_df_copy['date'] = pd.to_datetime(air_quality_df_copy['date']).dt.date

today = pd.to_datetime('today').date()

if today in air_quality_df_copy['date'].values:
    aq_today_df = air_quality_df.iloc[[-1]]    

if today not in air_quality_df_copy1['date'].values:
    # If not, insert the new data
    air_quality_fg.insert(aq_today_df)
    print(f"Data for {today} inserted successfully.")
else:
    print(f"Data for {today} already exists. No insertion performed.")

2024-11-21 09:30:51,569 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163414/fs/1154117/fg/1360206


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163414/jobs/named/air_quality_1_offline_fg_materialization/executions
Data for 2024-11-21 inserted successfully.


In [11]:
# Insert new data
#air_quality_fg.insert(aq_today_df)

In [12]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-21 09:21:06,560 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163414/fs/1154117/fg/1346955


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163414/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fa5604d66e0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670779
         }
       },
       "result": {
         "observed_value": 3.6712939739227295,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-21T09:21:06.000559Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_colum

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
